In [1]:
# Cell 1: Import libraries and load data
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Load original data
df = pd.read_csv('../data/creditcard.csv')
print("✅ Data loaded successfully!")
print(f"📊 Original dataset shape: {df.shape}")
print(f"   {df.shape[0]:,} transactions, {df.shape[1]} features")

✅ Data loaded successfully!
📊 Original dataset shape: (284807, 31)
   284,807 transactions, 31 features


In [2]:
# Cell 2: Create time-based features
print("🕐 CREATING TIME-BASED FEATURES")
print("="*60)

# Time is in seconds - convert to hours and days
df['Hour'] = (df['Time'] / 3600) % 24  # Hour of day (0-23)
df['Day'] = (df['Time'] / 86400).astype(int)  # Day number (0, 1, 2...)

print("✅ Created features:")
print("   - Hour: Hour of day (0-23)")
print("   - Day: Day number since start")

print(f"\n📊 Time range in dataset:")
print(f"   First transaction: {df['Time'].min()} seconds (Day 0, Hour 0)")
print(f"   Last transaction: {df['Time'].max()} seconds (Day {df['Day'].max()}, Hour {df['Hour'].iloc[-1]:.1f})")

# Show example
print(f"\n📝 Example transformation:")
sample = df[['Time', 'Hour', 'Day']].head(3)
print(sample)

🕐 CREATING TIME-BASED FEATURES
✅ Created features:
   - Hour: Hour of day (0-23)
   - Day: Day number since start

📊 Time range in dataset:
   First transaction: 0.0 seconds (Day 0, Hour 0)
   Last transaction: 172792.0 seconds (Day 1, Hour 24.0)

📝 Example transformation:
   Time      Hour  Day
0   0.0  0.000000    0
1   0.0  0.000000    0
2   1.0  0.000278    0


In [3]:
# Cell 3: Create amount-based features
print("💰 CREATING AMOUNT-BASED FEATURES")
print("="*60)

# Log transformation (handles outliers and skewness)
df['Amount_log'] = np.log1p(df['Amount'])  # log(1 + Amount) to handle zeros

# Polynomial feature (captures non-linear relationships)
df['Amount_squared'] = df['Amount'] ** 2

# Categorize amounts into bins
df['Amount_category'] = pd.cut(df['Amount'], 
                                bins=[0, 50, 150, 500, df['Amount'].max()],
                                labels=['Small', 'Medium', 'Large', 'Very_Large'])
df['Amount_category'] = df['Amount_category'].astype('category').cat.codes

print("✅ Created features:")
print("   - Amount_log: Log-transformed amount (handles outliers)")
print("   - Amount_squared: Polynomial feature (non-linear patterns)")
print("   - Amount_category: Categorized amounts (0=Small, 1=Medium, 2=Large, 3=Very Large)")

# Show statistics
print(f"\n📊 Amount feature statistics:")
print(f"   Original Amount - Mean: ${df['Amount'].mean():.2f}, Max: ${df['Amount'].max():.2f}")
print(f"   Log Amount - Mean: {df['Amount_log'].mean():.2f}, Max: {df['Amount_log'].max():.2f}")

# Show example
print(f"\n📝 Example transformation:")
sample = df[['Amount', 'Amount_log', 'Amount_squared', 'Amount_category']].head(5)
print(sample)

print(f"\n✅ New dataset shape: {df.shape}")
print(f"   Added {df.shape[1] - 31} new features!")

💰 CREATING AMOUNT-BASED FEATURES
✅ Created features:
   - Amount_log: Log-transformed amount (handles outliers)
   - Amount_squared: Polynomial feature (non-linear patterns)
   - Amount_category: Categorized amounts (0=Small, 1=Medium, 2=Large, 3=Very Large)

📊 Amount feature statistics:
   Original Amount - Mean: $88.35, Max: $25691.16
   Log Amount - Mean: 3.15, Max: 10.15

📝 Example transformation:
   Amount  Amount_log  Amount_squared  Amount_category
0  149.62    5.014760      22386.1444                1
1    2.69    1.305626          7.2361                0
2  378.66    5.939276     143383.3956                2
3  123.50    4.824306      15252.2500                1
4   69.99    4.262539       4898.6001                1

✅ New dataset shape: (284807, 36)
   Added 5 new features!


In [4]:
# Cell 4: Prepare features for modeling
print("🎯 PREPARING FEATURES FOR MODELING")
print("="*60)

# Separate features (X) and target (y)
X = df.drop(['Class'], axis=1)
y = df['Class']

print(f"✅ Separated data:")
print(f"   Features (X): {X.shape}")
print(f"   Target (y): {y.shape}")

# Identify features that need scaling
# V1-V28 are already scaled (PCA transformed)
# We need to scale: Time, Amount, Hour, Day, and new amount features
features_to_scale = ['Time', 'Amount', 'Hour', 'Day', 'Amount_log', 'Amount_squared']

print(f"\n📏 Features to be scaled: {len(features_to_scale)}")
for feat in features_to_scale:
    print(f"   - {feat}")

print(f"\n📊 Before scaling - sample values:")
print(X[features_to_scale].head(3))

🎯 PREPARING FEATURES FOR MODELING
✅ Separated data:
   Features (X): (284807, 35)
   Target (y): (284807,)

📏 Features to be scaled: 6
   - Time
   - Amount
   - Hour
   - Day
   - Amount_log
   - Amount_squared

📊 Before scaling - sample values:
   Time  Amount      Hour  Day  Amount_log  Amount_squared
0   0.0  149.62  0.000000    0    5.014760      22386.1444
1   0.0    2.69  0.000000    0    1.305626          7.2361
2   1.0  378.66  0.000278    0    5.939276     143383.3956


In [5]:
# Cell 5: Scale features
print("⚖️ SCALING FEATURES")
print("="*60)

# Initialize scaler (StandardScaler: mean=0, std=1)
scaler = StandardScaler()

# Scale the specified features
X[features_to_scale] = scaler.fit_transform(X[features_to_scale])

print("✅ Features scaled using StandardScaler")
print("   (Transformed to mean=0, standard deviation=1)")

print(f"\n📊 After scaling - sample values:")
print(X[features_to_scale].head(3))

print(f"\n📈 Scaling verification:")
for feat in features_to_scale:
    print(f"   {feat}: mean={X[feat].mean():.6f}, std={X[feat].std():.6f}")

print("\n💡 Note: Values close to 0 mean = good scaling!")

⚖️ SCALING FEATURES
✅ Features scaled using StandardScaler
   (Transformed to mean=0, standard deviation=1)

📊 After scaling - sample values:
       Time    Amount      Hour       Day  Amount_log  Amount_squared
0 -1.996583  0.244964 -2.486373 -0.983407    1.124303       -0.025978
1 -1.996583 -0.342475 -2.486373 -0.983407   -1.114639       -0.038095
2 -1.996562  1.160686 -2.486326 -0.983407    1.682368        0.039535

📈 Scaling verification:
   Time: mean=-0.000000, std=1.000002
   Amount: mean=-0.000000, std=1.000002
   Hour: mean=0.000000, std=1.000002
   Day: mean=0.000000, std=1.000002
   Amount_log: mean=0.000000, std=1.000002
   Amount_squared: mean=0.000000, std=1.000002

💡 Note: Values close to 0 mean = good scaling!


In [6]:
# Cell 6: Train-test split
print("✂️ SPLITTING DATA: TRAIN vs TEST")
print("="*60)

# Split with stratification (maintains fraud ratio in both sets)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,        # 30% for testing
    random_state=42,      # Reproducible results
    stratify=y            # Keep fraud ratio same in train and test
)

print(f"✅ Data split completed:")
print(f"\n📚 Training Set:")
print(f"   Total transactions: {len(X_train):,}")
print(f"   Frauds: {y_train.sum():,} ({y_train.sum()/len(y_train)*100:.3f}%)")
print(f"   Normal: {len(y_train) - y_train.sum():,} ({(len(y_train) - y_train.sum())/len(y_train)*100:.3f}%)")

print(f"\n🧪 Test Set:")
print(f"   Total transactions: {len(X_test):,}")
print(f"   Frauds: {y_test.sum():,} ({y_test.sum()/len(y_test)*100:.3f}%)")
print(f"   Normal: {len(y_test) - y_test.sum():,} ({(len(y_test) - y_test.sum())/len(y_test)*100:.3f}%)")

print("\n💡 Stratified split ensures fraud ratio is same in both sets!")
print("   This is CRITICAL for imbalanced datasets.")

✂️ SPLITTING DATA: TRAIN vs TEST
✅ Data split completed:

📚 Training Set:
   Total transactions: 199,364
   Frauds: 344 (0.173%)
   Normal: 199,020 (99.827%)

🧪 Test Set:
   Total transactions: 85,443
   Frauds: 148 (0.173%)
   Normal: 85,295 (99.827%)

💡 Stratified split ensures fraud ratio is same in both sets!
   This is CRITICAL for imbalanced datasets.


In [7]:
# Cell 7: Save processed data
print("💾 SAVING PROCESSED DATA")
print("="*60)

# Save to CSV files
X_train.to_csv('../data/X_train.csv', index=False)
X_test.to_csv('../data/X_test.csv', index=False)
y_train.to_csv('../data/y_train.csv', index=False)
y_test.to_csv('../data/y_test.csv', index=False)

print("✅ Files saved successfully:")
print("   ✓ ../data/X_train.csv")
print("   ✓ ../data/X_test.csv")
print("   ✓ ../data/y_train.csv")
print("   ✓ ../data/y_test.csv")

# Also save the full processed dataset (optional, for reference)
df.to_csv('../data/processed_full_data.csv', index=False)
print("   ✓ ../data/processed_full_data.csv (full dataset with new features)")

print(f"\n📊 Summary of saved data:")
print(f"   Training set: {X_train.shape[0]:,} transactions × {X_train.shape[1]} features")
print(f"   Test set: {X_test.shape[0]:,} transactions × {X_test.shape[1]} features")

print("\n🎉 PHASE 2 COMPLETE!")
print("="*60)
print("Next up: Phase 3 - Building Machine Learning Models! 🚀")

💾 SAVING PROCESSED DATA
✅ Files saved successfully:
   ✓ ../data/X_train.csv
   ✓ ../data/X_test.csv
   ✓ ../data/y_train.csv
   ✓ ../data/y_test.csv
   ✓ ../data/processed_full_data.csv (full dataset with new features)

📊 Summary of saved data:
   Training set: 199,364 transactions × 35 features
   Test set: 85,443 transactions × 35 features

🎉 PHASE 2 COMPLETE!
Next up: Phase 3 - Building Machine Learning Models! 🚀
